Vamos a construir un clasificador de spam or ham en python, utilizando todo lo que hemos aprendido en la lección.

In [8]:
# !wget https://lazyprogrammer.me/course_files/spam.csv
import pandas as pd
import sklearn as sk
import spacy
import re
import string
from sklearn.model_selection import train_test_split


In [9]:
# !wget https://lazyprogrammer.me/course_files/spam.csv

In [10]:
df_spam = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [11]:
df_spam.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Al inspeccionar nuestros datos por encima vemos que se han creado una serie de columnas cuya información no nos interesa. Debido a esto vamos a eliminarlas y quedarnos únicamente con la etiqueta y el texto. Además cambiaremos el nombre de estas columnas para poder visualizar y manejar mejor nuestros datos

In [12]:
for columna in df_spam.columns:
    if 'Unnamed' in columna:
        df_spam = df_spam.drop(columna, axis=1)

In [13]:
df_spam.head()
print(len(df_spam))

5572


In [14]:
df_spam = df_spam.rename(columns={'v1':'label','v2':'text'})

Ahora necesitamos una función de preprocesado que reduzca la dimensionalidad del modelo. Para ello vamos a escribir una función de normalización. Esta función lo que hará será pasar todo el texto a minúsculas y eliminar del mismo la puntuación. Después usando una función lambda vamos a aplicar dicha función a toda la columna del dataframe que nos interesa.

In [15]:
def normalize_text(raw_text):
    raw_text = str(raw_text).lower()
    text = raw_text.translate(str.maketrans('','',string.punctuation))
    return text

Esta línea de abajo funciona de la siguiente manera:

para la columna text del df_spam (que es un objeto de tipo Series) le decimos que, dentro de esa serie, a cada elemento x (de aquí viene el lambda x:) le aplique la función normalize_text.

Vamos a imprimir un elemento del df_spam para ver que la función se ha aplicado correctamente.


In [16]:
df_spam['text'] = df_spam['text'].apply(lambda x: normalize_text(x))

In [17]:
df_spam['text'][0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

Perfecto. Nuestros datos de entrenamiento están preparados. Ahora tenemos que hacer el paso siguiente: convertir las palabras a vectores. Para ello se pueden utilizar diferentes métodos. El countVectorizer podría ser una opción pero, dado que tenemos texto natural, es muy probable que las stopwrods y otras palabras sin sgnificado semántico tengan un peso muy superior a aquellas que podrían realmente identificar qué diferencia a spam de ham. Siendo así Hemos oprtado por el vectorizador TfIDF porque reduce el peso de las palabras que aparecen mucho en el dataset.

Al vectorizar creamos una matriz enorme en el que están las representaciones numéricas del lenguaje natural del dataset. Cada fila será un documento y cada coumna una palabra. Podemos ver en una celda superior como el primer número coincide en esta matriz con el length del dataframe (5572)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df_spam['text'])
vectors.shape

(5572, 9489)

Ahora vamos a dividir nuestros datos en sets de entrenamiento y test. train_test_split devuelve 4 elementos: 

1o -> datos de entrenamiento (en este caso vectores que usaremos para entrenar) -> X_train
2o -> etiquetas de datos de entrenamiento (indica si ese vector es spam o ham) -> X_test
3o -> datos de test (vectores de datos que usaremos para medir la performance) -> y_train
4o -> etiquetas de test (indica si el vector es spam o ham) -> y_test

In [19]:
X_train, X_test, y_train, y_test = train_test_split(vectors, df_spam['label'], test_size=0.15, random_state=111)

En este punto del código tenemos ya el dataframe procesado, los vecotres para cada uno de los textos junto a su etiqueta y hemos dividido en dos sets: uno de training y otro de test. Ahora tenemos que elegir un modelo. Esta parte es más complicada y requiere de visualización de datos al mismo tiempo que una comprensión más profunda de cómo funciona cada modelo. Tratándose de un ejercicio como este lo que vamos a hacer es probar diferentes clasificadores que vienen de serie de sklearn y ver qué resultados nos ofrecen.

In [20]:
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [21]:
# model = LogisticRegression(solver='liblinear', penalty='l1')
model = RandomForestClassifier(n_estimators=31, random_state=111)
# model = MultinomialNB(alpha=0.2)

Vamos a crear dos funciones muy básicas: la primera recibe el modelo, los vecotres de entrenamiento y las etiquetas para los datos. El segundo recibe el modelo entrenado y otros vecotres sobre el que hará predicciones. Después ejecutamos ambas funciones sorbe nuestros datos.

In [22]:
def train(variable_with_model, vectors_of_training, labels_of_training):
    variable_with_model.fit(vectors_of_training, labels_of_training)

def predict(variable_with_model_trained, new_vector):
    return (variable_with_model_trained.predict(new_vector))

In [23]:
train(model,X_train,y_train)
pred = predict(model,X_test)

Genial. Ya tenemos nuestro modelo de ML preparado y realizando predicciones. Ahora tenemos que ver cuan bien lo hace nuestro modelo. Empecemos por mirar la medida más básica: el accuracy

In [24]:
accuracy = accuracy_score(y_test,pred)
print(accuracy)

0.9760765550239234


Nuestro modelo tiene un accuracy de 96% lo que está bastante bien pero cuidado. No hemos mirado previamente (gran error) si las clases están o no balanceadas. Es decir si hay una muestra representativa suficiente de todas las posibilidades de modo que nuestro modelo pueda generalizar con precision. Necesitamos otras medidas. Para ello nos vamos a valer de lo que vimos en la clase sobre métricas, es decir: matrices de confusión, precision, recall y el F1 score. Estas técnicas nos servirán para ver realmente cuan bien lo está haciendo nuestro modelo.

In [25]:
from sklearn.metrics import confusion_matrix
y_pred_nb = model.predict(X_test)
y_true_nb = y_test
cm = confusion_matrix(y_true_nb, y_pred_nb)


Ya tenemos nuestra matriz de confusión que tiene el siguiente aspecto:
[[718   7]
 [ 25  86]]

Podríamos ahora implementar nosotros mismos las cuentas usando las fórmulas pero sklearn ya las incluye por lo que no habrá necesidad. Pero lo haremos igualmente para ver cuan diferente es

In [26]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

precision = cm[0][0] / (cm[0][0]+cm[1][0])
print(precision)
precision = precision_score(y_true_nb,y_pred_nb,average='weighted')
print(precision)
F1 = f1_score(y_true_nb,y_pred_nb,average='weighted')
print(F1)

0.9731543624161074
0.9767187951575094
0.9750549705883585


Como vemos la métrica sigue siendo bastante alta por lo que podemos confiar en que este modelo funciona bastante bien. Ya tenemos contruido nuestro clasificador de emailes entre spam o legítimo. De todos los modelos evaluados el que mejor métricas ofrece es el MultinomialNB por lo que en este caso sería el que elegiríamos.